In [1]:
import json
import random
import pickle
import numpy as np
from tqdm import tqdm
from pathlib import Path
import re
import math

## 1. 分析数据：是否限定同一个event mention，同一个event type，同一个trigger只会出现一次，而且它们之中的论元是固定的？

首先是检查是否出现同一个event mention，同一个event type，同一个trigger出现多次。

In [16]:
def analyze_trigger():
    data_dir = Path.cwd().parent.parent / "dataset"
    dataset = open(data_dir / "train_base.json","r")
    cnt = 0
    span_len_dict = dict()

    for idx, line in enumerate(dataset):
        example = json.loads(line)
        events = example['events']
        event_type_dict = dict()

        for event in events:
            type = event["type"]
            if type not  in event_type_dict:
                event_type_dict[type] = []

            for arg in event["mentions"]:
                if arg["role"] == "trigger":
                    span = arg["span"]
                    span_len = span[1]-span[0]
                    if span_len not in span_len_dict:
                        span_len_dict[span_len] = 0
                    span_len_dict[span_len] += 1
                    if span in event_type_dict[type]:
                        print(idx,type,span)
                    event_type_dict[type].append(span)

        for type in event_type_dict.keys():
            if len(event_type_dict[type]) > 1:
                #print(f"example-{idx} contains multiple triggers for event——{type}")
                cnt = cnt+1
    print(cnt)
    print(span_len_dict)

In [17]:
analyze_trigger()

4 股份股权转让 [21, 23]
8 质押 [60, 62]
17 股份股权转让 [135, 137]
17 股份股权转让 [135, 137]
20 减持 [27, 29]
27 股份股权转让 [55, 57]
27 股份股权转让 [70, 72]
27 股份股权转让 [70, 72]
27 股份股权转让 [70, 72]
27 股份股权转让 [70, 72]
30 投资 [43, 45]
30 投资 [43, 45]
30 投资 [43, 45]
30 投资 [43, 45]
30 投资 [43, 45]
30 投资 [43, 45]
30 投资 [43, 45]
39 投资 [23, 25]
39 投资 [23, 25]
39 投资 [23, 25]
39 股份股权转让 [23, 25]
39 股份股权转让 [23, 25]
39 股份股权转让 [23, 25]
39 股份股权转让 [80, 82]
39 股份股权转让 [80, 82]
87 减持 [50, 52]
87 减持 [50, 52]
87 减持 [50, 52]
87 减持 [50, 52]
87 减持 [50, 52]
87 投资 [11, 13]
87 投资 [11, 13]
87 投资 [38, 40]
87 股份股权转让 [11, 13]
87 股份股权转让 [11, 13]
87 股份股权转让 [38, 40]
87 股份股权转让 [50, 52]
87 股份股权转让 [50, 52]
87 股份股权转让 [50, 52]
87 股份股权转让 [50, 52]
87 股份股权转让 [50, 52]
98 股份股权转让 [139, 141]
127 股份股权转让 [37, 39]
127 股份股权转让 [56, 58]
133 投资 [15, 17]
133 投资 [15, 17]
133 股份股权转让 [2, 4]
133 股份股权转让 [2, 4]
148 股份股权转让 [63, 65]
164 股份股权转让 [36, 38]
164 股份股权转让 [36, 38]
170 质押 [38, 40]
177 股份股权转让 [46, 48]
177 股份股权转让 [46, 48]
177 股份股权转让 [46, 48]
177 股份股权转让 [46, 48]
177 股份股权转让 [46

2414 股份股权转让 [18, 20]
2414 股份股权转让 [18, 20]
2414 股份股权转让 [18, 20]
2414 股份股权转让 [18, 20]
2414 股份股权转让 [18, 20]
2414 股份股权转让 [18, 20]
2414 股份股权转让 [18, 20]
2414 股份股权转让 [18, 20]
2414 股份股权转让 [18, 20]
2414 股份股权转让 [18, 20]
2414 股份股权转让 [18, 20]
2414 股份股权转让 [18, 20]
2414 股份股权转让 [18, 20]
2425 投资 [76, 78]
2428 起诉 [37, 39]
2432 减持 [2, 4]
2432 股份股权转让 [2, 4]
2432 股份股权转让 [2, 4]
2441 投资 [26, 28]
2441 投资 [26, 28]
2445 减持 [16, 18]
2445 减持 [16, 18]
2445 减持 [16, 18]
2445 减持 [16, 18]
2457 股份股权转让 [30, 32]
2463 股份股权转让 [11, 13]
2469 股份股权转让 [22, 24]
2471 股份股权转让 [35, 37]
2471 股份股权转让 [35, 37]
2495 股份股权转让 [20, 22]
2495 股份股权转让 [103, 105]
2500 股份股权转让 [39, 41]
2501 股份股权转让 [15, 17]
2504 股份股权转让 [36, 38]
2512 减持 [5, 7]
2513 减持 [36, 38]
2513 投资 [20, 22]
2513 投资 [36, 38]
2513 投资 [59, 61]
2533 股份股权转让 [3, 5]
2536 投资 [31, 33]
2536 投资 [31, 33]
2544 投资 [16, 18]
2544 股份股权转让 [16, 18]
2546 质押 [87, 89]
2546 质押 [87, 89]
2546 质押 [87, 89]
2547 股份股权转让 [28, 30]
2556 股份股权转让 [32, 34]
2577 股份股权转让 [20, 22]
2578 股份股权转让 [21, 23]
2587 股份股权转让 [23, 

In [20]:
def analyze_args():
    data_dir = Path.cwd().parent.parent / "dataset"
    dataset = open(data_dir / "train_base.json","r")
    cnt = 0

    for idx, line in enumerate(dataset):
        example = json.loads(line)
        events = example['events']

        for event in events:
            type = event["type"]
            event_args_dict = dict()

            for arg in event["mentions"]:
                if arg["role"] != "trigger":
                    if arg["role"] not in event_args_dict:
                        event_args_dict[arg["role"]] = 1
                    else:
                        print(idx,type,arg["role"])
                        event_args_dict[arg["role"]] += 1
                        cnt += 1

    print(cnt)

In [21]:
analyze_args()

23 减持 sub
23 减持 sub
23 减持 sub
27 投资 obj
27 投资 obj
27 投资 obj
27 投资 obj
46 股份股权转让 target-company
55 减持 sub
55 股份股权转让 sub-per
61 减持 sub
61 减持 sub
63 投资 sub
63 股份股权转让 sub-org
81 起诉 obj-org
82 投资 obj
82 投资 obj
82 投资 obj
82 股份股权转让 target-company
82 股份股权转让 target-company
82 股份股权转让 target-company
96 减持 obj
96 减持 obj
96 减持 obj
96 减持 obj
96 减持 obj
96 减持 obj
96 减持 obj
102 股份股权转让 sub-org
108 减持 sub
108 减持 sub
108 股份股权转让 sub-per
136 投资 sub
136 投资 sub
136 投资 sub
142 减持 sub
149 减持 sub
152 减持 sub
153 质押 sub-per
161 质押 sub-per
162 股份股权转让 collateral
167 投资 sub
177 投资 obj
177 投资 obj
177 投资 obj
177 投资 obj
177 投资 obj
177 投资 obj
177 投资 obj
183 质押 sub-per
183 质押 sub-per
183 质押 sub-per
183 质押 sub-per
188 投资 sub
188 投资 sub
194 股份股权转让 obj-org
194 股份股权转让 collateral
202 起诉 sub-per
202 起诉 sub-per
205 股份股权转让 date
206 起诉 obj-org
206 起诉 obj-org
206 起诉 obj-org
206 起诉 obj-org
213 股份股权转让 money
214 减持 obj
214 减持 obj
215 减持 sub
219 投资 obj
228 减持 sub
228 减持 sub
229 起诉 obj-per
234 减持 sub
249 减持 sub
249 减持 sub
249 减持 sub
249

1105 起诉 sub-org
1105 起诉 sub-org
1107 股份股权转让 target-company
1112 质押 sub-org
1115 起诉 sub-org
1115 起诉 sub-org
1115 起诉 obj-org
1116 股份股权转让 obj-org
1129 投资 sub
1129 投资 obj
1134 投资 obj
1134 投资 obj
1134 投资 obj
1134 投资 obj
1134 投资 obj
1134 投资 obj
1134 投资 obj
1134 股份股权转让 obj-org
1134 股份股权转让 target-company
1134 股份股权转让 target-company
1134 股份股权转让 target-company
1134 股份股权转让 target-company
1134 股份股权转让 target-company
1134 股份股权转让 target-company
1134 股份股权转让 target-company
1135 减持 sub
1135 减持 sub
1135 减持 sub
1140 投资 sub
1140 股份股权转让 obj-org
1156 减持 sub
1156 减持 title
1159 投资 obj
1163 投资 obj
1181 减持 sub
1181 减持 title
1181 股份股权转让 sub-per
1183 起诉 obj-org
1183 起诉 obj-org
1183 起诉 obj-org
1189 投资 obj
1203 起诉 obj-org
1209 投资 sub
1213 股份股权转让 date
1214 减持 sub
1216 起诉 obj-org
1216 起诉 obj-org
1216 起诉 obj-org
1234 减持 sub
1244 质押 obj-org
1248 减持 title
1250 起诉 obj-org
1253 股份股权转让 sub-per
1253 股份股权转让 sub-per
1253 股份股权转让 sub-per
1253 股份股权转让 sub-per
1257 减持 sub
1257 减持 sub
1261 投资 obj
1261 投资 obj
1264 投资 obj
1264 股份股权转让 t

2266 股份股权转让 sub-org
2268 起诉 sub-per
2269 起诉 obj-per
2271 股份股权转让 money
2271 股份股权转让 proportion
2277 减持 share-org
2277 减持 obj
2281 股份股权转让 target-company
2281 股份股权转让 target-company
2286 投资 obj
2288 投资 sub
2290 投资 obj
2305 起诉 obj-org
2306 股份股权转让 collateral
2307 减持 sub
2308 减持 obj
2309 股份股权转让 target-company
2313 质押 sub-per
2314 股份股权转让 obj-org
2314 股份股权转让 target-company
2320 起诉 sub-org
2324 股份股权转让 obj-org
2324 股份股权转让 target-company
2324 股份股权转让 target-company
2324 股份股权转让 target-company
2331 起诉 sub-org
2331 起诉 obj-org
2332 投资 sub
2343 投资 sub
2343 股份股权转让 obj-org
2349 起诉 obj-org
2352 减持 obj
2368 减持 obj
2370 股份股权转让 obj-org
2370 股份股权转让 obj-org
2371 质押 obj-org
2371 质押 obj-org
2376 减持 sub
2384 投资 sub
2384 股份股权转让 obj-org
2387 起诉 sub-org
2387 起诉 obj-org
2387 起诉 obj-org
2395 投资 obj
2395 股份股权转让 collateral
2395 股份股权转让 target-company
2397 股份股权转让 money
2402 起诉 obj-per
2404 股份股权转让 sub-org
2404 股份股权转让 sub-org
2404 股份股权转让 obj-org
2404 股份股权转让 obj-org
2414 减持 sub
2414 减持 sub
2414 减持 sub
2414 减持 sub
2414 减持 sub
2